In [54]:
import os
import pickle
import itertools
from datetime import datetime
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_pinball_loss

os.chdir("C:/2023_11-PTSFC")
import model_train as model_train
import data_prepro as data_prepro
import model_eval as model_eval

In [55]:
import importlib
importlib.reload(model_eval)

<module 'model_eval' from 'C:\\2023_11-PTSFC\\model_eval.py'>

### Import

In [56]:
# os.environ["LOKY_MAX_CPU_COUNT"] = "1"  # Replace "4" with the desired number of cores

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
fcast_hor = [36, 40, 44, 60, 64, 68] # in hours

# = = = = = = = = = = = = = 
# get data
# df_energy = data_prepro.get_energy_data_today(to_date=t_wednesday.strftime('%Y%m%d'))

# Read data from file with specified data types
df_energy = pd.read_csv("data/2015-01-01_2024-02-21_energy.csv", index_col=0, parse_dates=[0])
df_energy['timestamp_CET'] = pd.to_datetime(df_energy['timestamp_CET'], utc=True).dt.tz_convert('CET')
print(df_energy.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80136 entries, 2014-12-31 23:00:00+00:00 to 2024-02-21 22:00:00+00:00
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp_CET  80136 non-null  datetime64[ns, CET]
 1   gesamt         80136 non-null  float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 1.8 MB
None


In [67]:
# Read Pickle File of Scores
with open("2024-03-28_14-29-58 final all models/eval.pickle", "rb") as f:
    scores_dict = pickle.load(f)

# Read Pickle File of Fcasts
with open("2024-03-28_14-29-58 final all models/fcasts.pickle", "rb") as f:
    fcasts_dict = pickle.load(f)

### All Models Evaluation

In [68]:
input_dict = scores_dict
weekly_scores_df_out = pd.DataFrame(columns=input_dict.keys(),
                                    index=list(input_dict[list(input_dict.keys())[0]].keys()))

for week_key, weekly_scores in input_dict.items():

    for key, model_scores in weekly_scores.items():
        
        # print(model_scores)

        # entire mean
        model_qscore_mean = model_scores.values.mean()

        # print(f"> {key}: {model_qscore_mean}")
        weekly_scores_df_out.loc[key, week_key] = model_qscore_mean

ranking_df = weekly_scores_df_out.rank(axis=0, method='min')
ranking_sum = ranking_df.sum(axis=1)
ranking_sum = ranking_sum.sort_values(ascending=True)
print("> ranking sum")
pprint(ranking_sum.head(5))

mean_qscores_df = weekly_scores_df_out.mean(axis=1).sort_values(ascending=True)
print("> qscore sum")
pprint(mean_qscores_df.head(10))

> ranking sum
xgboost_dummy_2020      92.0
xgboost_dummy_2021      99.0
mstl_0.5               102.0
xgboost_dummy_2019     104.0
lightgbm_dummy_2020    123.0
dtype: float64
> qscore sum
mstl_0.5                 0.752816
mstl_1                   0.800466
xgboost_dummy_2021        0.82362
xgboost_dummy_2020       0.828085
xgboost_dummy_2019       0.860353
lightgbm_dummy_2020      1.030033
bench_pm_1month          1.044384
grad_boost_2018_dummy    1.053459
lightgbm_dummy_2021      1.064932
grad_boost_2018_fturs    1.070209
dtype: object


### Ensemble Combinations ...

In [69]:
fcasts_dict_all = fcasts_dict
all_models = list(fcasts_dict_all['2023-11-15'].keys())
print(all_models)

['bench_pm_2weeks', 'bench_same_month', 'bench_pm_1month', 'mstl_1', 'mstl_0.5', 'quant_reg_2015_sk', 'quant_reg_2018_sk', 'quant_reg_2015_sm', 'quant_reg_2018_sm', 'grad_boost_2015_fturs', 'grad_boost_2018_fturs', 'grad_boost_2015_dummy', 'grad_boost_2018_dummy', 'lightgbm_dummy_2019', 'lightgbm_dummy_2020', 'lightgbm_dummy_2021', 'xgboost_dummy_2019', 'xgboost_dummy_2020', 'xgboost_dummy_2021']


In [70]:
all_models = list(fcasts_dict_all['2023-11-15'].keys())
# Generate all possible combinations
ens_size = 3
combis = list(itertools.combinations(all_models, ens_size))
# filter out all ensembles with repeating models
combis = [combi for combi in combis if len(set(combi)) == ens_size]
print(len(combis))

# Define the start and end dates
start_date = pd.Timestamp('2023-11-15')
end_date = pd.Timestamp('2024-02-14')

# Generate a list of weekly dates in UTC
fcast_dates_cet = pd.date_range(start=start_date, end=end_date, freq='W-WED').tz_localize('CET').strftime('%Y-%m-%d').tolist()

all_combi_names = [f"ensemble | {', '.join(combi)}" for combi in combis]
print(all_combi_names[0])
res_df = pd.DataFrame(index=all_combi_names, columns=fcast_dates_cet)

# Iterate over the forecast dates
for fcast_idx, fcast_date in enumerate(fcast_dates_cet):

    print('= '*30)
    print(f"Forecasting for week starting from {fcast_date} ...")

    # = = = = = = = = = = = = = 
    # generate prediction timestamps based on t0 = following thursday 00:00
    # = = = = = = = = = = = = = 

    # Calculate the Thursday and Wednesday of the week
    t_wednesday = pd.Timestamp(fcast_date).replace(hour=0, minute=0, second=0, microsecond=0).tz_localize('CET')
    t_thursday = t_wednesday + pd.Timedelta(days=1)

    # Generate required submission timestamps
    subm_timestamps = [(t_thursday + pd.Timedelta(hours=fcast)) for fcast in fcast_hor]
    print(f"Submission timestamps = {subm_timestamps[0]} to {subm_timestamps[-1]}")

    weekly_fcasts = fcasts_dict_all[fcast_date]

    # = = = = = = = = = = = = = 
    # Evaluation based on submission timestamps
    # = = = = = = = = = = = = = 

    # get actual values at every submission timestamp
    df_energy_eval = df_energy.loc[df_energy['timestamp_CET'].isin(subm_timestamps)].copy()

    for combi_idx, combi in enumerate(combis):

        # calculate ensemble between the preds of the models in the current combination
        pred_list = [weekly_fcasts[model] for model in combi]
        new_name = f"ensemble | {', '.join(combi)}"

        # Ignore timestamp_CET column and take the average of the quantiles
        pred_vals_list = [pred.iloc[:, 1:].copy() for pred in pred_list]

        # Take the average of the quantiles across all models in the ensemble
        ens_pred_df = pred_list[0].copy()
        ens_pred_df.iloc[:, 1:] = sum(pred_vals_list) / len(pred_vals_list)
        
        df_scores = model_eval.eval_fcast_qscore(ens_pred_df, df_energy_eval, subm_timestamps, quantiles)

        # Save the last row of the scores dataframe to final output
        res_df.loc[new_name, fcast_date] = df_scores.values.mean()


969
ensemble | bench_pm_2weeks, bench_same_month, bench_pm_1month
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
Forecasting for week starting from 2023-11-15 ...
Submission timestamps = 2023-11-17 12:00:00+01:00 to 2023-11-18 20:00:00+01:00


In [61]:
res_df.mean(axis=1).sort_values(ascending=True).head(20)

ensemble | mstl_0.5, grad_boost_2018_fturs, xgboost_dummy_2021    1.214518
ensemble | mstl_0.5, grad_boost_2018_dummy, xgboost_dummy_2021    1.221724
ensemble | mstl_0.5, grad_boost_2015_dummy, xgboost_dummy_2021     1.23793
ensemble | mstl_0.5, grad_boost_2015_fturs, xgboost_dummy_2021    1.239091
ensemble | mstl_0.5, xgboost_dummy_2019, xgboost_dummy_2021       1.243077
ensemble | mstl_1, mstl_0.5, xgboost_dummy_2021                   1.247709
ensemble | mstl_1, mstl_0.5, grad_boost_2015_dummy                1.251296
ensemble | mstl_1, mstl_0.5, grad_boost_2015_fturs                1.251526
ensemble | mstl_0.5, grad_boost_2015_fturs, xgboost_dummy_2019    1.254158
ensemble | bench_pm_2weeks, mstl_0.5, xgboost_dummy_2021          1.258444
ensemble | mstl_0.5, grad_boost_2015_dummy, xgboost_dummy_2019    1.260238
ensemble | mstl_1, grad_boost_2018_fturs, xgboost_dummy_2021       1.26158
ensemble | mstl_1, mstl_0.5, grad_boost_2018_dummy                1.265321
ensemble | mstl_1, grad_b

In [62]:
# Create Own Ensembles
model_list = ['mstl_0.5', 'bench_pm_1month', 'lightgbm_dummy_2020', 'xgboost_dummy_2021']

model_scores = np.zeros(len(fcast_dates_cet))                
# Iterate over the forecast dates
for fcast_idx, fcast_date in enumerate(fcast_dates_cet):

    # Calculate the Thursday and Wednesday of the week
    t_wednesday = pd.Timestamp(fcast_date).replace(hour=0, minute=0, second=0, microsecond=0).tz_localize('CET')
    subm_timestamps = [(t_wednesday + pd.Timedelta(days=1, hours=fcast)) for fcast in fcast_hor]
    print(f"Submission timestamps = {subm_timestamps[0]} to {subm_timestamps[-1]}")

    # get actual values at every submission timestamp
    df_energy_eval = df_energy.loc[df_energy['timestamp_CET'].isin(subm_timestamps)].copy()
    # get weekly fcast
    weekly_fcasts = fcasts_dict_all[fcast_date]

    # calculate ensemble between the preds of the models in the current combination
    pred_list = [weekly_fcasts[model] for model in model_list]
    new_name = f"ensemble | {', '.join(combi)}"

    # Ignore timestamp_CET column and take the average of the quantiles
    pred_vals_list = [pred.iloc[:, 1:].copy() for pred in pred_list]

    # Take the average of the quantiles across all models in the ensemble
    ens_pred_df = pred_list[0].copy()
    ens_pred_df.iloc[:, 1:] = sum(pred_vals_list) / len(pred_vals_list)

    df_scores = model_eval.eval_fcast_qscore(ens_pred_df, df_energy_eval, subm_timestamps, quantiles)
    model_scores[fcast_idx] = df_scores.values.mean()

Submission timestamps = 2023-11-17 12:00:00+01:00 to 2023-11-18 20:00:00+01:00


KeyError: 'lightgbm_dummy_2020_5'

In [ ]:
model_scores.mean()

1.2579703451958013